In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns


In [3]:

data = pd.read_csv('EVChargingStationUsage.csv')

C:\Users\s153529\AppData\Local\Temp\ipykernel_27176\422452095.py:1: DtypeWarning: Columns (29,30,32) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv('EVChargingStationUsage.csv')


In [4]:
#function to extract the entity based on the first word following "PALO ALTO CA /"
def extract_entity(station_name):
    parts = station_name.split('/')
    if len(parts) > 1:
        # Take the part after "PALO ALTO CA /" and then take the first word
        return parts[1].strip().split(' ')[0]
    else:
        # If the station name does not follow the expected format, return the original
        return station_name

# Function to create lagged features
def buildLaggedFeatures(s, lag=30, dropna=True):
    df = pd.concat([s.shift(i) for i in range(lag + 1)], axis=1)
    df.columns = ['lag_{}'.format(i) if i != 0 else s.name for i in range(lag + 1)]
    if dropna:
        df = df.dropna()
    return df


In [5]:
data["Normalized Station Name"] = data["Station Name"].str.split(pat=" / | #| [0-9]").apply(lambda col: col[1])
data = data.dropna(subset=['Normalized Station Name'])

In [6]:
stations = data['Normalized Station Name'].unique()

In [7]:
data.head()

,Station Name,MAC Address,Org Name,Start Date,Start Time Zone,End Date,End Time Zone,Transaction Date (Pacific Time),Total Duration (hh:mm:ss),Charging Time (hh:mm:ss),...,Currency,Fee,Ended By,Plug In Event Id,Driver Postal Code,User ID,County,System S/N,Model Number,Normalized Station Name
0,PALO ALTO CA / HAMILTON #1,000D:6F00:015A:9D76,City of Palo Alto,7/29/2011 20:17,PDT,7/29/2011 23:20,PDT,7/29/2011 23:20,3:03:32,1:54:03,...,USD,0.0,Plug Out at Vehicle,3,95124.0,3284.0,NaN,NaN,NaN,HAMILTON
1,PALO ALTO CA / HAMILTON #1,000D:6F00:015A:9D76,City of Palo Alto,7/30/2011 0:00,PDT,7/30/2011 0:02,PDT,7/30/2011 0:02,0:02:06,0:01:54,...,USD,0.0,Customer,4,94301.0,4169.0,NaN,NaN,NaN,HAMILTON
2,PALO ALTO CA / HAMILTON #1,000D:6F00:015A:9D76,City of Palo Alto,7/30/2011 8:16,PDT,7/30/2011 12:34,PDT,7/30/2011 12:34,4:17:32,4:17:28,...,USD,0.0,Plug Out at Vehicle,5,94301.0,4169.0,NaN,NaN,NaN,HAMILTON
3,PALO ALTO CA / HAMILTON #1,000D:6F00:015A:9D76,City of Palo Alto,7/30/2011 14:51,PDT,7/30/2011 16:55,PDT,7/30/2011 16:55,2:03:24,2:02:58,...,USD,0.0,Customer,6,94302.0,2545.0,NaN,NaN,NaN,HAMILTON
4,PALO ALTO CA / HAMILTON #1,000D:6F00:015A:9D76,City of Palo Alto,7/30/2011 18:51,PDT,7/30/2011 20:03,PDT,7/30/2011 20:03,1:11:24,0:43:54,...,USD,0.0,Plug Out at Vehicle,7,94043.0,3765.0,NaN,NaN,NaN,HAMILTON


In [8]:
def secondTransform(x: str) -> int:
    h, m, s = x.split(":")
    return int(h) * 3600 + int(m) * 60 + int(s)

data["Total Duration s"] = data["Total Duration (hh:mm:ss)"].apply(lambda x: secondTransform(x))
data["Charging Time s"] = data["Charging Time (hh:mm:ss)"].apply(lambda x: secondTransform(x))
data["Idle Time s"] = data['Total Duration s']-data["Charging Time s"]


In [9]:
data["Transaction Date (Pacific Time)"] = pd.to_datetime(data["Transaction Date (Pacific Time)"], format="%m/%d/%Y %H:%M", errors='coerce')
data.dropna(subset=['Transaction Date (Pacific Time)'], inplace=True)

In [10]:
piv_data = pd.pivot_table(data, values="Energy (kWh)", index="Transaction Date (Pacific Time)", columns= "Normalized Station Name", aggfunc=np.sum, fill_value=0)
piv_data.reset_index(inplace=True)

In [11]:
df = piv_data.assign(timestamp=pd.to_datetime(piv_data["Transaction Date (Pacific Time)"]).dt.date).groupby('timestamp', as_index=False).sum()

In [12]:
plotting_data = df.iloc[30:]

In [13]:
plotting_data[["timestamp", "HAMILTON"]]

Normalized Station Name,timestamp,HAMILTON
30,2011-08-28,24.390405
31,2011-08-29,34.526345
32,2011-08-30,24.120799
33,2011-08-31,39.792270
34,2011-09-01,27.460373
...,...,...
3440,2020-12-29,0.000000
3441,2020-12-30,0.000000
3442,2020-12-31,0.000000
3443,2021-01-01,0.000000


In [ ]:
plt.plot(plotting_data["HAMILTON"])
plt.show()

In [2]:
plt.plot(plotting_data["MPL"])
plt.show()

NameError: name 'plt' is not defined

In [ ]:
#fig, ax = plt.subplots(1,1, figsize=(10,5))
#sns.lineplot(ax=ax, data=plotting_data, x="timestamp", y="HAMILTON")
plt.plot(plotting_data["timestamp"].dt.strftime('%Y-%m-%d'), plotting_data["HAMILTON"])
plt.show()

In [ ]:

# Training
lagged_features_hamilton = buildLaggedFeatures_off(daily_energy_hamilton, lag=30, offset=30)
train_data_hamilton = lagged_features_hamilton.iloc[:-30]  
test_data_hamilton = lagged_features_hamilton.iloc[-30:]   